# Data Scraping

In [2]:
from bs4 import BeautifulSoup
import requests
q=input("Enter the stock symbol: ")
url=f"https://merolagani.com/CompanyDetail.aspx?symbol={q}"
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
}
response=requests.get(url,headers=headers)
soup=BeautifulSoup(response.content,'html.parser')
if response.status_code == 200:
    table=soup.find_all('tr')
    for row in table:
        print(row.get_text(strip=True))
        '''columns=row.find_all(['th','td'])
        for column in columns:
            data=column.text.strip().replace('\n','').replace('\r','')
            print(data,end=' ')'''
        print(" ")
else:
    print("Failed to retrieve data from the website.")

SectorCommercial Banks
 
Shares Outstanding270,569,970.00
 
Market Price499.30
 
% Change0.65 %
 
Last Traded On2026/02/05 02:59:54
 
52 Weeks High - Low562.00-471.00
 
180 Day Average506.96
 
120 Day Average502.04
 
1 Year Yield1.90%
 
EPS35.18(FY:082-083, Q:2)
 
P/E Ratio14.19
 
Book Value235.64
 
PBV2.12
 
% Dividend12.50(FY:081-082)
 
#Fiscal YearValue1.12.50%(FY: 081-082)2.10.00%(FY: 080-081)3.11.00%(FY: 079-080)4.0.00%(FY: 079-080)5.11.50%(FY: 078-079)6.0.00%(FY: 078-079)7.4.40%(FY: 077-078)8.1.76%(FY: 076-077)9.0.00%(FY: 075-076)10.0.00%(FY: 074-075)11.18.00%(FY: 073-074)12.15.00%(FY: 072-073)13.6.84%(FY: 071-072)14.0.00%(FY: 070-071)15.40.00%(FY: 069-070)16.40.00%(FY: 068-069)17.30.00%(FY: 067-068)
 
#Fiscal YearValue
 
1.12.50%(FY: 081-082)
 
2.10.00%(FY: 080-081)
 
3.11.00%(FY: 079-080)
 
4.0.00%(FY: 079-080)
 
5.11.50%(FY: 078-079)
 
6.0.00%(FY: 078-079)
 
7.4.40%(FY: 077-078)
 
8.1.76%(FY: 076-077)
 
9.0.00%(FY: 075-076)
 
10.0.00%(FY: 074-075)
 
11.18.00%(FY: 073-074)
 
12

# Cleaning Data

In [9]:
data=[]
for row in table:
    columns=row.find_all(['th','td'])
    for column in columns:
        data.append(column.text.strip().replace('\n','').replace('\r',''))
#print(data)
clean_data={}
i=0
while i < len(data)-1:
    key=data[i].strip()
    value=data[i+1].strip()    
    if key and value and not key.startswith("#") and not key.startswith("% Bonus") and not key in ["", "Fiscal Year", "Value"]:
        clean_data[key]=value
    i+=2
print(clean_data)



{'Sector': 'Commercial Banks', 'Shares Outstanding': '270,569,970.00', 'Market Price': '499.30', '% Change': '0.65 %', 'Last Traded On': '2026/02/05 02:59:54', '52 Weeks High - Low': '562.00-471.00', '180 Day Average': '506.96', '120 Day Average': '502.04', '1 Year Yield': '1.90%', 'EPS': '35.18                                                                    (FY:082-083, Q:2)', 'P/E Ratio': '14.19', 'Book Value': '235.64', 'PBV': '2.12', '% Dividend': '12.50                                                                    (FY:081-082)', '1.': '12.50%', '(FY: 081-082)': '2.', '10.00%': '(FY: 080-081)', '3.': '11.00%', '(FY: 079-080)': '4.', '0.00%': '(FY: 070-071)', '5.': '11.50%', '(FY: 078-079)': '6.', '7.': '4.40%', '(FY: 077-078)': '8.', '1.76%': '(FY: 076-077)', '9.': '0.00%', '(FY: 075-076)': '10.', '11.': '18.00%', '(FY: 073-074)': '12.', '15.00%': '(FY: 072-073)', '13.': '6.84%', '(FY: 071-072)': '14.', '15.': '40.00%', '(FY: 069-070)': '16.', '40.00%': '(FY: 068-069)', '17

# Database Connection

In [39]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

con = psycopg2.connect(
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT"),
    user=os.getenv("DB_USER"),

    password=os.getenv("DB_PASSWORD"),
    database=os.getenv("DB_NAME")
)
cur=con.cursor()
cur.execute("CREATE TABLE IF NOT EXISTS stock_data (id SERIAL PRIMARY KEY, stock_symbol VARCHAR(10), MarketPrice VARCHAR(20), PercentageChange VARCHAR(20), YearlyHighLow VARCHAR(40))")
cur.execute("INSERT INTO stock_data (stock_symbol, MarketPrice, PercentageChange,YearlyHighLow) VALUES (%s, %s,%s, %s)", (q, clean_data["Market Price"], clean_data["% Change"], clean_data["52 Weeks High - Low"]))
con.commit()
cur.execute("SELECT * FROM stock_data")
rows = cur.fetchall()   
for row in rows:
    print(row)

(8, 'nabil', '499.30', '0.65 %', '562.00-471.00')


In [38]:
cur.execute("DELETE FROM stock_data")
con.commit()

In [40]:
cur.close()
con.close() 